<a href="https://colab.research.google.com/github/amoyag/Biofisica/blob/main/S4_lambdaswitch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Negative feedback loop. The Lambda Phage Switch

## **Introducción**
*Nota: Es conveniente que amplíes este resumen teórico con el capítulo 5 de An Introduction to Systems Biology de Uri Alon.*

Los sistemas biológicos pueden decidir entre diferentes estados, haciéndolos permanentes. El bacteriófago lambda, un virus que infecta bacterias *E. coli*, es un buen ejemplo de este proceso; cuando el fago infecta una célula, debe “elegir” entre la **vía lítica**, en la que se reproduce rápidamente y destruye la célula huésped, o la **vía lisogénica**, en la que se integra en el genoma bacteriano y permanece latente.

Esta elección está controlada por un circuito molecular centrado en dos proteínas clave: **CI** (el represor λ) y **Cro**, que se reprimen mutuamente. De este modo tenemos un **interruptor biológico**, que establece un estado: niveles altos de CI mantienen el estado lisogénico, mientras que niveles altos de Cro desencadenan la vía lítica.

Lo que hace que este sistema sea especialmente interesante es su **naturaleza biestable**. Igual que un interruptor de luz que solo puede estar “encendido” o “apagado”, el interruptor lambda tiende a estabilizarse en uno de dos estados posibles, sin posiciones intermedias. Esta biestabilidad surge de la represión mutua entre CI y Cro, combinada con la **unión cooperativa** (las proteínas se unen como dímeros) y con tasas cuidadosamente ajustadas de producción y degradación.

En esta simulación podrás explorar cómo funciona este interruptor, modelando las interacciones moleculares clave mediante un sistema de ODEs. El modelo incorpora las características:

*   Unión cooperativa de proteínas (representada por coeficientes de Hill).
*   Represión mutua mediante unión a operadores.
*   Producción basal (“expresión con fuga”).
*   Degradación proteica.

Al ajustar parámetros como las tasas de producción (β), las tasas de degradación (α), las constantes de unión (K) y las condiciones iniciales, podremos investigar:

*   Cómo pequeñas diferencias en los niveles iniciales de proteínas conducen a resultados radicalmente distintos.
*   Las condiciones necesarias para mantener la bistabilidad.
*   El papel de la cooperatividad en la creación de un comportamiento tipo interruptor.
*   Cómo la degradación proteica afecta la estabilidad de cada estado.




## Banco de módulos

In [4]:
#@title Módulo 0. Librerías
# -*- coding: utf-8 -*-
# =========================
# Módulo 0: Librerías y estilo
# =========================
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.integrate import solve_ivp
import ipywidgets as widgets
from IPython.display import display

plt.style.use('classic')
sns.set_theme(style="whitegrid")

# Paleta consistente
PALETTE = {
    "CI": "#1f77b4",   # azul
    "Cro": "#d62728",  # rojo
    "nullcline_CI": "#1f77b4",
    "nullcline_Cro": "#d62728",
    "vector_field": "#7f7f7f"
}


In [5]:
#@title Módulo 1. Modelo Lambda Switch
# =========================
# Módulo 1: Modelo del Lambda Switch
# =========================
def hill_repression(x, K, n):
    """
    Función de represión tipo Hill: g(x) = 1 / (1 + (x/K)^n)
    x: concentración del represor
    K: constante de semisaturación
    n: coeficiente de Hill (cooperatividad)
    """
    return 1.0 / (1.0 + (x / K)**n)

def model_lambda_switch(t, y, params):
    """
    Modelo mínimo del interruptor lambda con represión mutua y expresión basal.

    Variables:
      y = [CI, Cro]

    Ecuaciones:
      dCI/dt  = beta_CI * g_Cro(Cro) + beta0_CI - alpha_CI * CI
      dCro/dt = beta_Cro * g_CI(CI)  + beta0_Cro - alpha_Cro * Cro

    Donde g_X(.) es la función de represión Hill del represor correspondiente.

    params esperados:
      params["CI"]  = {"beta": ..., "beta0": ..., "alpha": ..., "K": ..., "n": ...}
      params["Cro"] = {"beta": ..., "beta0": ..., "alpha": ..., "K": ..., "n": ...}
      (opcional) params["input"] = función u(t) para modular producción (ver abajo)
      (opcional) params["input_effect"] = dict con claves "CI" y/o "Cro" y ganancia k

    Nota: Si se desea entrada externa, se implementa como modulación multiplicativa
          de la producción máxima: beta_eff = beta * (1 + k * u(t)).
    """
    CI, Cro = y

    # Parám. CI
    beta_CI  = params["CI"]["beta"]
    beta0_CI = params["CI"]["beta0"]
    alpha_CI = params["CI"]["alpha"]
    K_CI     = params["CI"]["K"]
    n_CI     = params["CI"].get("n", 2)

    # Parám. Cro
    beta_Cro  = params["Cro"]["beta"]
    beta0_Cro = params["Cro"]["beta0"]
    alpha_Cro = params["Cro"]["alpha"]
    K_Cro     = params["Cro"]["K"]
    n_Cro     = params["Cro"].get("n", 2)

    # Entrada externa opcional (p. ej., señal de daño SOS)
    u = 0.0
    if "input" in params and callable(params["input"]):
        u = params["input"](t)  # Sintaxis correcta: función de entrada evaluada en t

    # Modulación opcional de betas por entrada externa
    if "input_effect" in params:
        k_CI  = params["input_effect"].get("CI", 0.0)
        k_Cro = params["input_effect"].get("Cro", 0.0)
        beta_CI_eff  = beta_CI  * (1.0 + k_CI  * u)
        beta_Cro_eff = beta_Cro * (1.0 + k_Cro * u)
    else:
        beta_CI_eff, beta_Cro_eff = beta_CI, beta_Cro

    # Represiones Hill
    g_Cro_on_CI  = hill_repression(Cro, K_Cro, n_Cro)  # Cro reprime CI
    g_CI_on_Cro  = hill_repression(CI,  K_CI,  n_CI)   # CI  reprime Cro

    # Ecuaciones dinámicas
    dCI_dt  = beta_CI_eff  * g_Cro_on_CI + beta0_CI  - alpha_CI  * CI
    dCro_dt = beta_Cro_eff * g_CI_on_Cro + beta0_Cro - alpha_Cro * Cro

    return [dCI_dt, dCro_dt]


In [6]:
#@title Módulo 2. Config

# =========================
# Módulo 2: Configuración
# =========================
def build_config(input_type="none", t_span=(0.0, 200.0), y0=None):
    """
    Crea la configuración por defecto del interruptor lambda.

    input_type: "none" | "step" | "pulse"
      - "none": sin entrada externa.
      - "step": u(t)=1 para t>=t_on.
      - "pulse": u(t)=1 en [t_on, t_on+dur].

    t_span: tupla (t0, tf)
    y0: condiciones iniciales [CI0, Cro0]; si es None, usa [0.1, 0.1]
    """
    # Entrada externa (opcional)
    if input_type == "none":
        input_func = lambda t: 0.0
    elif input_type == "step":
        t_on = 20.0
        input_func = lambda t, t_on=t_on: 1.0 if t >= t_on else 0.0
    elif input_type == "pulse":
        t_on, dur = 20.0, 30.0
        input_func = lambda t, t_on=t_on, dur=dur: 1.0 if (t >= t_on and t <= t_on + dur) else 0.0
    else:
        raise ValueError("input_type debe ser 'none', 'step' o 'pulse'.")

    # Parámetros base (ajustables en los ejercicios)
    params = {
        "input": input_func,
        # Ganancias de entrada externa (por defecto 0: sin efecto)
        "input_effect": {"CI": 0.0, "Cro": 0.0},
        "CI":  {"beta": 2.0, "beta0": 0.05, "alpha": 0.2, "K": 1.0, "n": 2},
        "Cro": {"beta": 2.0, "beta0": 0.05, "alpha": 0.2, "K": 1.0, "n": 2},
    }

    if y0 is None:
        y0 = [0.1, 0.1]

    return params, t_span, y0

In [7]:
#@title Módulo 3. Simulación

# =========================
# Módulo 3: Simulación
# =========================
def run_simulation(model_func, params, t_span, y0, t_eval=None, rtol=1e-6, atol=1e-9):
    """
    Ejecuta la simulación con solve_ivp y devuelve tiempos y trayectorias.
    """
    if t_eval is None:
        t_eval = np.linspace(t_span[0], t_span[1], 2000)

    sol = solve_ivp(
        model_func, t_span, y0,
        args=(params,),
        t_eval=t_eval, rtol=rtol, atol=atol, method="RK45"
    )
    return sol.t, sol.y


In [8]:
#@title Módulo 4. Visualización

# =========================
# Módulo 4: Visualización (series temporales, nulclinas, campo de fases)
# =========================
def plot_timeseries(t, y, title="Dinámica temporal del interruptor lambda"):
    """
    Dibuja las series temporales de CI y Cro.
    """
    CI, Cro = y[0], y[1]
    plt.figure(figsize=(8, 5))
    plt.plot(t, CI,  color=PALETTE["CI"],  label="CI")
    plt.plot(t, Cro, color=PALETTE["Cro"], label="Cro")
    plt.xlabel("Tiempo")
    plt.ylabel("Concentración")
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()

def compute_nullclines(params, ci_range, cro_range):
    """
    Calcula nulclinas analíticas aproximadas:
      - Nulclina CI: dCI/dt=0 => CI = (beta_CI_eff*g_Cro + beta0_CI)/alpha_CI
      - Nulclina Cro: Cro = (beta_Cro_eff*g_CI + beta0_Cro)/alpha_Cro

    Devuelve arrays (CI_nc, Cro_nc) discretizados sobre las rejillas cro_range y ci_range respectivamente.
    """
    # Extrae parámetros
    beta_CI  = params["CI"]["beta"]
    beta0_CI = params["CI"]["beta0"]
    alpha_CI = params["CI"]["alpha"]
    K_Cro    = params["Cro"]["K"]
    n_Cro    = params["Cro"].get("n", 2)

    beta_Cro  = params["Cro"]["beta"]
    beta0_Cro = params["Cro"]["beta0"]
    alpha_Cro = params["Cro"]["alpha"]
    K_CI      = params["CI"]["K"]
    n_CI      = params["CI"].get("n", 2)

    # Suponemos sin modulación externa para las nulclinas (u(t) ~ 0)
    beta_CI_eff, beta_Cro_eff = beta_CI, beta_Cro

    # Nulclina de CI en función de Cro
    g_Cro = 1.0 / (1.0 + (cro_range / K_Cro)**n_Cro)
    CI_nc = (beta_CI_eff * g_Cro + beta0_CI) / alpha_CI

    # Nulclina de Cro en función de CI
    g_CI = 1.0 / (1.0 + (ci_range / K_CI)**n_CI)
    Cro_nc = (beta_Cro_eff * g_CI + beta0_Cro) / alpha_Cro

    return CI_nc, Cro_nc

def plot_phase_portrait(model_func, params, bounds=(0, 3, 0, 3), grid=25, t_arrows=0.0):
    """
    Dibuja el campo de fases (vector field) y las nulclinas sobre el plano (CI, Cro).
    bounds: (CI_min, CI_max, Cro_min, Cro_max)
    grid: densidad de rejilla para el campo de fases
    t_arrows: tiempo al que evaluar la entrada externa si se desea (0 por defecto)
    """
    CI_min, CI_max, Cro_min, Cro_max = bounds
    ci = np.linspace(CI_min, CI_max, grid)
    cro = np.linspace(Cro_min, Cro_max, grid)
    CI_mesh, Cro_mesh = np.meshgrid(ci, cro)

    # Campo de fases
    dCI = np.zeros_like(CI_mesh)
    dCro = np.zeros_like(Cro_mesh)
    for i in range(grid):
        for j in range(grid):
            dydt = model_func(t_arrows, [CI_mesh[i, j], Cro_mesh[i, j]], params)
            dCI[i, j], dCro[i, j] = dydt

    # Nulclinas
    CI_nc, Cro_nc = compute_nullclines(params, ci_range=ci, cro_range=cro)

    plt.figure(figsize=(7, 6))
    # Vector field
    plt.quiver(CI_mesh, Cro_mesh, dCI, dCro, color=PALETTE["vector_field"], angles='xy', scale_units='xy', scale=1.5, alpha=0.6, width=0.003)
    # Nulclina CI: CI vs Cro
    plt.plot(CI_nc, cro, color=PALETTE["nullcline_CI"],  lw=2, label="dCI/dt = 0")
    # Nulclina Cro: CI vs Cro
    plt.plot(ci, Cro_nc, color=PALETTE["nullcline_Cro"], lw=2, label="dCro/dt = 0")

    plt.xlabel("CI")
    plt.ylabel("Cro")
    plt.xlim(CI_min, CI_max)
    plt.ylim(Cro_min, Cro_max)
    plt.title("Campo de fases y nulclinas (interruptor lambda)")
    plt.legend()
    plt.tight_layout()
    plt.show()


In [9]:
#@title Módulo 5. Análisis

# =========================
# Módulo 5: Análisis (estado final, cuencas de atracción)
# =========================
def classify_state(CI_final, Cro_final, ratio_thresh=1.0):
    """
    Clasifica el estado final:
      - 'lysogeny' si CI_final / Cro_final >= ratio_thresh
      - 'lysis'     si CI_final / Cro_final <  ratio_thresh

    ratio_thresh=1.0 implica comparación directa CI vs Cro.
    """
    eps = 1e-9
    ratio = (CI_final + eps) / (Cro_final + eps)
    return "lisogenia" if ratio >= ratio_thresh else "lisis"

def compute_outcome(t, y, settle_window=0.1):
    """
    Devuelve estado final y valores finales promediados al término de la simulación.
    settle_window: fracción final del intervalo temporal para promediar (por defecto 10%).
    """
    N = max(5, int(len(t) * settle_window))
    CI_final = float(np.mean(y[0][-N:]))
    Cro_final = float(np.mean(y[1][-N:]))
    state = classify_state(CI_final, Cro_final)
    return state, CI_final, Cro_final

def basin_of_attraction(model_func, params, t_span, ci0_range, cro0_range, steps=25, t_eval=None):
    """
    Explora cuencas de atracción: para una rejilla de condiciones iniciales (CI0, Cro0),
    simula y clasifica el estado final. Devuelve:
      - grid_CI0, grid_Cro0 (mallas)
      - outcome_matrix (0=lisis, 1=lisogenia)
    """
    ci0_vals  = np.linspace(ci0_range[0],  ci0_range[1],  steps)
    cro0_vals = np.linspace(cro0_range[0], cro0_range[1], steps)
    grid_CI0, grid_Cro0 = np.meshgrid(ci0_vals, cro0_vals)

    outcome_matrix = np.zeros_like(grid_CI0, dtype=int)

    for i in range(steps):
        for j in range(steps):
            y0 = [grid_CI0[i, j], grid_Cro0[i, j]]
            t, y = run_simulation(model_func, params, t_span, y0, t_eval=t_eval)
            state, _, _ = compute_outcome(t, y)
            outcome_matrix[i, j] = 1 if state == "lisogenia" else 0

    return grid_CI0, grid_Cro0, outcome_matrix

def plot_basin(grid_CI0, grid_Cro0, outcome_matrix, cmap="RdBu", title="Cuencas de atracción"):
    """
    Dibuja el mapa de cuencas de atracción:
      rojo -> lisis (0)
      azul -> lisogenia (1)
    """
    plt.figure(figsize=(7, 6))
    im = plt.imshow(
        outcome_matrix, origin="lower", aspect="auto",
        extent=[grid_CI0.min(), grid_CI0.max(), grid_Cro0.min(), grid_Cro0.max()],
        cmap=cmap, vmin=0, vmax=1
    )
    plt.colorbar(im, label="Estado final (0=lisis, 1=lisogenia)")
    plt.xlabel("CI₀ (condición inicial)")
    plt.ylabel("Cro₀ (condición inicial)")
    plt.title(title)
    plt.tight_layout()
    plt.show()


In [10]:
#@title Módulo 6. Utilidades

# =========================
# Módulo 6: Utilidades (entrada, helpers)
# =========================
def make_step(t_on=20.0, amplitude=1.0):
    """
    Genera una entrada escalón: u(t) = amplitude para t >= t_on; 0 en otro caso.
    """
    return lambda t, t_on=t_on, amplitude=amplitude: amplitude if t >= t_on else 0.0

def make_pulse(t_on=20.0, duration=30.0, amplitude=1.0):
    """
    Genera una entrada pulso: u(t) = amplitude en [t_on, t_on+duration]; 0 en otro caso.
    """
    return lambda t, t_on=t_on, duration=duration, amplitude=amplitude: (
        amplitude if (t >= t_on and t <= t_on + duration) else 0.0
    )

def copy_params(params):
    """
    Copia segura del diccionario de parámetros (copia superficial por claves de primer nivel
    y copia por clave para subdiccionarios CI y Cro).
    """
    new_params = {}
    for k, v in params.items():
        if isinstance(v, dict):
            new_params[k] = {kk: vv for kk, vv in v.items()}
        else:
            new_params[k] = v
    return new_params


## Ejercicios


### Ejercicio 1. Visualización de la biestabilidad y dinámica temporal**

**Objetivo:**  
Explorar cómo el interruptor del fago lambda presenta dos estados estables (lisogenia y lisis) y cómo la dinámica temporal depende de los parámetros del sistema.




**Instrucciones:**

1.  Ajusta los parámetros
    *   **$\beta_{CI}$** y **$\beta_{Cro}$**: tasas máximas de producción de CI y Cro.
    *   **$\alpha_{CI}$** y **$\alpha_{Cro}$**: tasas de degradación.
    *   **$\beta_{oCI}$** y **$\beta_{oCro}$**: expresión basal.
2.  Observa:
    *   Las curvas temporales de CI y Cro.
    *   El **campo de fases** con las nulclinas y el vector field.
3.  Analiza:
    *   ¿Qué ocurre si aumentas la tasa de producción de CI?
    *   ¿Cómo cambia la posición de los atractores?
    *   ¿Por qué el sistema se estabiliza en uno de dos estados y no en un punto intermedio?

El campo de fases es una herramienta para visualizar cómo evoluciona el sistema en el plano formado por las dos variables: CI y Cro. Cada punto representa un estado posible (una combinación de concentraciones), y las flechas indican la dirección y velocidad del cambio en ese punto.
Las nulclinas son curvas donde la derivada de una variable es cero:

- $\frac{dCI}{dt} = 0$ (línea azul): puntos donde CI no cambia.
- $\frac{dCro}{dt} = 0$ (línea roja): puntos donde Cro no cambia.

Los puntos donde ambas nulclinas se cruzan son puntos fijos:

Si los vectores alrededor apuntan hacia el cruce, es estable (atractor).
Si se alejan, es inestable (repulsor).

El campo de fases muestra la estructura global del sistema: hacia dónde se mueve desde cada región y qué atractores existen.

**Preguntas para reflexionar:**

*   ¿Qué parámetros favorecen la lisogenia? ¿Cuáles favorecen la lisis?
*   ¿Por qué la biestabilidad es importante para la toma de decisiones biológicas?

***




In [12]:

# =========================
# Ejercicio 1: Bistabilidad y dinámica temporal (Interactivo)
# =========================
import ipywidgets as widgets
from IPython.display import display

# Sliders para parámetros clave
beta_CI_slider  = widgets.FloatSlider(value=2.0, min=0.5, max=5.0, step=0.1, description='β_CI')
beta_Cro_slider = widgets.FloatSlider(value=2.0, min=0.5, max=5.0, step=0.1, description='β_Cro')
alpha_CI_slider = widgets.FloatSlider(value=0.2, min=0.05, max=0.5, step=0.01, description='α_CI')
alpha_Cro_slider= widgets.FloatSlider(value=0.2, min=0.05, max=0.5, step=0.01, description='α_Cro')
beta0_CI_slider = widgets.FloatSlider(value=0.05, min=0.0, max=0.2, step=0.01, description='β₀_CI')
beta0_Cro_slider= widgets.FloatSlider(value=0.05, min=0.0, max=0.2, step=0.01, description='β₀_Cro')

# Función interactiva
def simulate_bistability(beta_CI, beta_Cro, alpha_CI, alpha_Cro, beta0_CI, beta0_Cro):
    # Configuración base
    params, t_span, y0 = build_config(input_type="none", t_span=(0, 200), y0=[0.1, 0.1])

    # Ajustar parámetros según sliders
    params["CI"]["beta"]  = beta_CI
    params["CI"]["alpha"] = alpha_CI
    params["CI"]["beta0"] = beta0_CI
    params["Cro"]["beta"]  = beta_Cro
    params["Cro"]["alpha"] = alpha_Cro
    params["Cro"]["beta0"] = beta0_Cro

    # Simulación
    t, y = run_simulation(model_lambda_switch, params, t_span, y0)

    # Visualización: series temporales
    plot_timeseries(t, y, title="Dinámica temporal: CI vs Cro")

    # Visualización: campo de fases y nulclinas
    plot_phase_portrait(model_lambda_switch, params, bounds=(0, 6, 0, 6), grid=25)

    # Estado final
    state, CI_final, Cro_final = compute_outcome(t, y)
    print(f"Estado final: {state}")
    print(f"CI final ≈ {CI_final:.2f}, Cro final ≈ {Cro_final:.2f}")

# Crear interfaz interactiva
widgets.interactive(simulate_bistability,
    beta_CI=beta_CI_slider,
    beta_Cro=beta_Cro_slider,
    alpha_CI=alpha_CI_slider,
    alpha_Cro=alpha_Cro_slider,
    beta0_CI=beta0_CI_slider,
    beta0_Cro=beta0_Cro_slider
)

interactive(children=(FloatSlider(value=2.0, description='β_CI', max=5.0, min=0.5), FloatSlider(value=2.0, des…